In [1]:
import pickle
from keras.applications.inception_v3 import InceptionV3
encode_model=InceptionV3(weights='imagenet')
encode_model.layers.pop()

Using TensorFlow backend.


In [2]:
for layer in encode_model.layers:
    layer.trainable=False
print(encode_model.summary())

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
_______________________________________________________________________________________

In [3]:
from keras import Model
model_output = encode_model.get_layer("avg_pool").output
model = Model(inputs=encode_model.input, outputs=model_output)

In [4]:
import cv2
import os
import numpy as np

#f = 'C:\\Users\\Rahat Santosh\\Desktop\\dataset_trial\\'
f = '../img'
x={}
for filename in os.listdir(f):
    i=0
    full_path = os.path.join(f, filename)
    y=[]
    if full_path=="../img/.ipynb_checkpoints":
        continue
    if os.path.isdir(full_path):
        for fg in os.listdir(full_path):
            im = os.path.join(full_path, fg)
            if im.endswith(".jpg"):
                imgs=cv2.imread(im,cv2.IMREAD_COLOR)
                new=cv2.resize(imgs,(299,299))
                a=model.predict(np.expand_dims(new,0))
                y.append([a])
        y=np.reshape(y,(np.shape(y)[0],np.shape(y)[3]))
        x[filename]=y
print(len(x))

4


In [5]:
print(np.shape(x['ic']))
print(x['ic'])
print(len(x['ic']))
print(len(x))
n=len(x['ic'])
l=len(x)
print(n)
print(l)

(124, 2048)
[[ 4.912944    0.9838035   9.156374   ...  0.          0.
   0.        ]
 [ 0.4971447   0.0955028   0.10147487 ...  0.48437616  0.
  10.113127  ]
 [ 2.1254928   1.0651304   2.7344193  ...  0.          0.11315354
   9.160039  ]
 ...
 [11.826998    0.23583186  0.12555455 ...  0.          0.
  10.482667  ]
 [ 4.072661    1.918773    1.6314207  ...  0.          0.
   1.8120645 ]
 [ 6.6977415   2.069224    0.         ...  0.          0.
   0.        ]]
124
4
124
4


In [6]:
m=0
for y in x.values():
    m=m+np.shape(y)[0]
print(m)
size=m*n*2
print(m*n*2)

496
123008


In [7]:
data=np.zeros((size,3,2048))
y1=np.empty((size,),dtype=np.int_)
i=0
for (label,y) in x.items():
    for a in y:
        data[i:i+n,0,:]=a
        data[i:i+n,1,:]=y
        y1[i:i+n]=1
        i=i+n
        
j=0
for (label,y) in x.items():
    k=['ab3','ic','library','shops']
    del k[k.index(label)]
    for a in y:
        data[i:i+n,0,:]=a
        data[i:i+n,1,:]=x[k[j]]
        y1[i:i+n]=0
        i=i+n
        j=(j+1)%(l-1)
        
print(i)

123008


In [ ]:
z=np.sqrt(np.square(np.subtract(data[:,0,:],data[:,1,:])))
data[:,2,:]=z
print(z)
print(np.shape(z))

In [ ]:
new=[data,y1]

In [10]:
import random
random.shuffle(new)

In [11]:
data=new[1]
y1=new[0]
print(np.shape(data))
print(np.shape(y1))

(12168, 3, 2048)
(12168,)


In [12]:
new=[]
for (label,y) in x.items():
    for a in y:
        new.append([a,label])
print(np.shape(new[0][0]))
print(np.shape(new))
print(new[0][1])

(2048,)
(156, 2)
shops


In [13]:
print(new)

[[array([0.        , 8.097372  , 0.13073428, ..., 0.        , 0.        ,
       0.26550743], dtype=float32), 'shops'], [array([0.        , 4.740665  , 0.        , ..., 0.05128904, 0.        ,
       1.7649796 ], dtype=float32), 'shops'], [array([0.        , 3.1897683 , 0.        , ..., 0.        , 0.11589367,
       0.18717921], dtype=float32), 'shops'], [array([0.02051763, 3.7797894 , 0.        , ..., 0.58908665, 0.        ,
       1.4942373 ], dtype=float32), 'shops'], [array([0.06810052, 6.629798  , 0.        , ..., 0.        , 0.        ,
       0.24617817], dtype=float32), 'shops'], [array([0.23327884, 3.9386475 , 0.11883484, ..., 0.        , 0.        ,
       0.6656349 ], dtype=float32), 'shops'], [array([ 0.13912554, 10.339495  ,  0.08360869, ...,  0.07297845,
        0.        ,  0.26144916], dtype=float32), 'shops'], [array([1.2134786 , 8.228564  , 0.        , ..., 0.05813516, 0.        ,
       1.7288184 ], dtype=float32), 'shops'], [array([0.3793854 , 6.9953985 , 0.4057850

In [14]:
pickle_out=open('one_shot_pre_data.pickle','wb')
pickle.dump(data,pickle_out)
pickle_out.close()

pickle_out=open('one_shot_pre_label.pickle','wb')
pickle.dump(y1,pickle_out)
pickle_out.close()

In [15]:
model.save('encode_model.h5')

In [16]:
pickle_out=open('one_shot_pre_data_2.pickle','wb')
pickle.dump(z,pickle_out)
pickle_out.close()

In [17]:
pickle_out=open('one_shot_inputs.pickle','wb')
pickle.dump(new,pickle_out)
pickle_out.close()